In [15]:
import pymystem3

In [16]:
import re

In [9]:
!pip install transliterate

In [17]:
import transliterate

In [18]:
#эта часть кода открывает твой текст, лемматизирует его по определенному принципу (здесь принцип -c - он сохраняет оригинал и при этом рядом пишет лемму)
#и сохраняет его в новом файле, который стоит заранее создать. 
#вначале надо писать путь к программе mystem, потом путь к редактируемому тексту, потом путь к пустому файлу, куда все сохранится
!C:\Users\Elisaveta\MyStem\mystem.exe C:\Users\Elisaveta\Documents\GitHub\Coursework-2020\Clean_2_level.txt C:\Users\Elisaveta\Documents\GitHub\Coursework-2020\text.txt -c 

In [41]:
f = open(r'text.txt', encoding = "UTF-8") #открываем кусок дневника
text = f.read()
f.close()
dnevnik = text.split(' ') #сплитим кусок по пробелам и видим слово и его лемму

In [20]:
f = open(r"Surnames russian.txt", "r", encoding = "UTF-8") #открываем файл со списком
                                                                                                         #русских фамилий без инициалов
list1 = f.readlines()
f.close()
list_of_surnames = []
for w in list1:
    w = w.lower() #приводим фамилии к нижнему регистру
    if w:
        list_of_surnames.append(w.strip()) #добавляем это все в новый список
        
list_of_surnames_english = [] #создаем пустой список для английского транслита
for n in list_of_surnames: #обращаемся к списку с фамилиями на русском
    if n:
        name = transliterate.translit(n, reversed=True) #транслитерируем русскую фамилию
        list_of_surnames_english.append(name) #добавляем в пустой список, где должен быть транлит
        

dict_surnames = dict(zip(list_of_surnames, list_of_surnames_english)) #соединяем два списка - с фамилиями на рус и англ, получая таким образом словарь

#with open(r".\Dictionary of surnames.txt", "w") as fw:
  #  for word in d:
  #      fw.write("{} \t {}\n". format(word, d[word]))

In [21]:
def surname_2_mark(surname, original_form):
    return '<persName ref="#'+ dict_surnames[surname] +'" lemma = "'+ surname +'" #>'+ original_form +'</persName>'

In [45]:
reg = "\{\S*\}" #это регулярка для того, что находится в {} скобках

marked_text= []
for string in dnevnik: #работаем с переменной, в которой у нас список со словами и их леммами
    #print ('исходная строка ', string)
    allregs = re.findall(reg, string) #находим в списке подстроки, которые подходят под регулярку
    if allregs: #если нашли такую подстроку, начинаем ее очищать от всякой грязи
        for word in allregs: 
            original_form = re.sub(reg, '', string) #очищаем строки от их лемм и сохраняем в таком виде
            #print ()
            word = word.strip('{}')
            word = word.replace("|", " ")
            word = word.replace("?","")
            
            if ' ' in word and re.match ('[А-ЯЁ]', re.sub('[^А-ЯЁа-яё]','',original_form)): #смотрим на те строки, где у нас подряд написано несколько имен и при этом проверяем, начинаются ли исходные строки, а не леммы, с большой буквы
                list_words = word.split() #превращаем их в список из нескольких элементов
                printed = False
                for n in list_words:
                    n.strip() #каждый элемент списка как бы чистим, превращая таким образом это в строки, которые в двух разных линиях
                    if n in dict_surnames: #ищем подстроки в словаре
                        #print ('case 1')
                        marked_text.append(surname_2_mark(n, original_form))
                        
                        printed = True
                if not printed:
                    #print ('case 2')
                    marked_text.append(original_form)
                   
            elif word in dict_surnames and re.match ('[А-ЯЁ]', re.sub('[^А-ЯЁа-яё]','',original_form)): #возвращаемся к нормальным одиноким словам в строке, ищем их в словаре
                #print ('case 3')
                marked_text.append(surname_2_mark(word, original_form)) #если программа находит, пишет тэг
            
            else:
                #print ('case 4')
                marked_text.append(original_form) #пишем итоговый вариант
    else:
        marked_text.append(string)
    
new_text = ' '.join(marked_text)

In [23]:
#Проблемы, которые возникли при разметке: 
#омонимия фамилий (белый/Белый: Жизнь Арсеньева и Арсеньев как реальный человек); 
#что делать с прозвищами/именами, как их соединить с фамилиями
#как соединить все в одну строку

#Идеи:
#можно разметить каждый день дневника (придумать регулярки, потому что формат у датирования всегда разный), и потом попросить
#программу выдавать только строки, которые начинаются с <>. Предварительно, скорее всего, надо все почистить (от той же омонимии)

#Что надо сделать:
#очистить словарь от инициалов

In [49]:
final_text = open(r'./Marked_text.txt', 'w')
final_text.write(new_text)
final_text.close()